Fetch information about a specific person identified by his/her ORCID ID, and handle API errors.

[Download Notebook](https://github.com/researchgraph/augment-api-beta/blob/main/docs/notebooks/orcid.ipynb)

In [ ]:
import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ast
import altair as alt
import networkx as nx
import nx_altair as nxa
from wordcloud import WordCloud, STOPWORDS 
import pandas as pd
from datetime import datetime, date
import requests
import json

import os
from os.path import join, dirname
from dotenv import load_dotenv
load_dotenv();

### ORCID ID not found

In [ ]:
# ORCID ID not found
API_KEY = os.environ.get("API_KEY")
ORCID = "0000-0003-XXXX-XXXX"

url = f'https://api.researchgraph.ai/v1/orcid/{ORCID}?subscription-key={API_KEY}'
r = requests.get(url)

# print a short confirmation on completion
print('Augment API query complete ', r.status_code)

if r.status_code == 400:
    print(r.json()["message"])

### Missing API_KEY

In [ ]:
# Missing API_KEY
API_KEY = ''
ORCID = "0000-0002-0068-716X"

url = f'https://api.researchgraph.ai/v1/orcid/{ORCID}?subscription-key={API_KEY}'
r = requests.get(url)

# print a short confirmation on completion
print('Augment API query complete ', r.status_code)

if r.status_code == 401:
    print(f'Authentication error.')

### ORCID ID does exist

In [ ]:
# ORCID ID does exist
API_KEY = os.environ.get("API_KEY")
ORCID = "0000-0002-0068-716X"

url = f'https://api.researchgraph.ai/v1/orcid/{ORCID}?subscription-key={API_KEY}'
r = requests.get(url)

# print a short confirmation on completion
print('Augment API query complete ', r.status_code)

if r.status_code == 200 and r.json()[0]["nodes"]["researchers"]:    
    researchers = r.json()[0]["nodes"]["researchers"]
    
    researcher = None
    for i in range(len(researchers)):
        if researchers[i]["orcid"] == ORCID:
            researcher = researchers[i]

    print()
    print(f'ORCID: {researcher["orcid"]}')
    print(f'First name: {researcher["first_name"]}')
    print(f'Last name: {researcher["last_name"]}')
    print()
    print(f'The researcher {researcher["full_name"]} is connected to {r.json()[0]["stats"]}.')

### List of co-authors
Only includes co-authors with ORCID IDs.

In [ ]:
rf = pd.DataFrame(r.json()[0]["nodes"]["researchers"], columns=['first_name', 'last_name', 'full_name', 'orcid'])
dfStyler = rf.style.set_properties(**{'text-align': 'left'})
dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

In [ ]:
# Fetch relationships between all co-authors
API_KEY = os.environ.get("API_KEY")

# Generate a graph from the co-authors
G = nx.Graph()

# Convert from and to into ORCID IDs (to map the node labels)
def force_orcid(n):
    n['from'] = n['from'].split('/')[-1]
    n['to'] = n['to'].split('/')[-1]
    return n

for a in rf['orcid']:
    url = f'https://api.researchgraph.ai/v1/orcid/{a}?subscription-key={API_KEY}'
    r = requests.get(url)

    # print a short confirmation on completion
    print('Augment API query complete ', r.status_code)
    
    json = map(force_orcid, r.json()[0]['relationships']['researcher-researcher'])
    ef = pd.DataFrame(json, columns=['from', 'to'])
    ef = ef[ef['from'].isin(rf['orcid'].to_list())]
    G.add_edges_from(ef.to_numpy())

In [ ]:
for index, row in rf.iterrows():
    G.add_node(row['orcid'], name=row['full_name'], color='#54C48C')

json = map(force_orcid, r.json()[0]['relationships']['researcher-researcher'])
ef = pd.DataFrame(json, columns=['from', 'to'])
G.add_edges_from(ef.to_numpy())

# Compute positions
pos = nx.spring_layout(G)

options = {
    "font_size": 12,
    "node_size": 50,
    "edge_color": "lightgray",
    "node_color": "#54C48C",
    "linewidths": 0.1,
    "width": 1
}

# Show information about the graph
print(nx.info(G))
print("Network density:", nx.density(G))

# export graph to a gephi file
nx.write_gexf(G, "co-authors.gexf")

# Draw the graph using altair
viz = nxa.draw_networkx(G, pos=pos, node_tooltip='name', **options).properties(width=800, height=800)
viz.interactive()